In [5]:
#Bryan René Rojas Hernández 
import json
import requests
from azure.cosmos import exceptions, CosmosClient, PartitionKey
import datetime

# Información de Azure Cosmos DB
ENDPOINT = 'https://cosmosbryan.documents.azure.com:443/'  # URL de punto de conexión de Cosmos DB
KEY = 'gUbmXMZF3Y3ybK1kynlQWvxO4Ag0ORybbEdhKUXz27JsrAQsOFVf4u5yNojfaqtFP8fXLrlrP7ItACDbjA69MQ=='  # Clave de acceso a Cosmos DB
DATABASE_ID = 'cosmosbryan'  # Identificador de la base de datos en Cosmos DB
CONTAINER_ID = 'COSMOS_DATABASE'  # Identificador del contenedor en Cosmos DB
PARTITION_KEY = '/ip'  # Ruta de la clave de partición en Cosmos DB

# Conectarse a Cosmos DB
client = CosmosClient(ENDPOINT, KEY)
database = client.get_database_client(DATABASE_ID)

# Verificar si la base de datos ya existe, de lo contrario, créala
database_client = None
try:
    #lee la información de la base de datos
    database_client = database.read()

# Si la base de datos no existe, captura la excepción
except exceptions.CosmosResourceNotFoundError:
    # Crea la base de datos con el ID especificado
    database_client = client.create_database(id=DATABASE_ID)

# creo un contenedor dentro de la base de datos
container_definition = {'id': CONTAINER_ID, 'partition_key': {'paths': [PARTITION_KEY], 'kind': 'Hash'}}
container_options = {'offerThroughput': 400}  # Puedes ajustar la cantidad de rendimiento según tus necesidades

# Obtener el cliente del contenedor de la base de datos (si ya existe)
container = database.get_container_client(CONTAINER_ID)
try:
    # Intenta leer la información del contenedor
    container_client = container.read()

# Si el contenedor no existe, captura la excepción CosmosResourceNotFoundError
except exceptions.CosmosResourceNotFoundError:
    # Crea el contenedor con el ID y configuración especificados
    container_client = database.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path=PARTITION_KEY),
                                                offer_throughput=container_options['offerThroughput'])


def is_ip_already_stored(container, ip):
    # Verificar si la IP ya está almacenada en Cosmos DB
    query = f"SELECT * FROM c WHERE c.ip = '{ip}'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return len(items) > 0

def is_ip_vulnerable(ip, respuesta_json):
    # Verificar si hay eventos relacionados con la IP objetivo y si son vulnerables
    eventos_filtrados = [evento for evento in respuesta_json.get('events', [])
                         if evento.get('target', {}).get('ip') == ip and
                         evento.get('result', {}).get('data', {}).get('status') == 'VULNERABLE']

    return len(eventos_filtrados) > 0

def insert_ip_info(container, ip, es_vulnerable):
    # Verificar si la IP ya está almacenada
    if is_ip_already_stored(container, ip):
        # Si la IP ya está en la base de datos, imprime un mensaje y sale de la función.
        print(f"La IP {ip} ya está almacenada en Cosmos DB.")
        return

    # Crear un documento con la información de la IP
    document = {
        'id': f'IP_{ip}',  # El identificador único del documento, se compone de 'IP_' seguido de la dirección IP.
        'ip': ip,  # La dirección IP que se está procesando.
        'vulnerable': es_vulnerable,  # Indica si la IP es vulnerable o no (booleano).
        'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Marca de tiempo actual.
    }

    # Insertar el documento en Cosmos DB
    container.create_item(body=document)  # Utiliza el cliente de contenedor para insertar el documento en Cosmos DB.
    print(f"La IP {ip} ha sido almacenada en Cosmos DB.")  # Mensaje indicando que la operación fue exitosa.


def consultar_ips(container):
    print('\nConsulta IPs almacenadas\n')

    # Consultar todas las IPs
    items = list(container.query_items(
        query="SELECT * FROM r",
        enable_cross_partition_query=True
    ))

    for item in items:
        print(f'IP: {item["ip"]}, Vulnerable: {item["vulnerable"]}, Timestamp: {item["timestamp"]}')
#Función API
def main():
    # Definir la URL de la API de BinaryEdge para buscar información sobre Bluekeep en una IP específica
    url = 'https://api.binaryedge.io/v2/query/search?query=type:bluekeep%20AND%20ip:134.175.237.77'

    # Establecer encabezados, incluido el token de autenticación (X-Key)
    headers = {'X-Key': '2286877a-3109-476c-b581-31985dbada5e'}

    # Realizar una solicitud GET a la API de BinaryEdge para obtener información sobre la IP y Bluekeep
    respuesta_api = requests.get(url, headers=headers)

    # Convertir la respuesta de la API a formato JSON
    respuesta_json = respuesta_api.json()

    # IP objetivo que estamos evaluando
    ip_objetivo = '134.175.237.77'

    # Verificar si la IP es vulnerable a Bluekeep utilizando la función is_ip_vulnerable
    es_vulnerable = is_ip_vulnerable(ip_objetivo, respuesta_json)

    # Imprimir el resultado basado en si la IP es vulnerable o no
    if es_vulnerable:
        print(f"La IP {ip_objetivo} es vulnerable a Bluekeep.")
    else:
        print(f"La IP {ip_objetivo} no es vulnerable a Bluekeep.")

    # Insertar información en Cosmos DB si la IP no está almacenada previamente
    insert_ip_info(container, ip_objetivo, es_vulnerable)

    # Consultar todas las IPs almacenadas en Cosmos DB
    consultar_ips(container)

# La siguiente línea ejecuta la función main() solo si este script es ejecutado como el programa principal
if __name__ == '__main__':
    main()



La IP 134.175.237.77 no es vulnerable a Bluekeep.
La IP 134.175.237.77 ya está almacenada en Cosmos DB.

Consulta IPs almacenadas

IP: 185.198.75.90, Vulnerable: True, Timestamp: 2024-03-06 08:46:26
IP: 80.120.238.18, Vulnerable: True, Timestamp: 2024-03-06 08:50:05
IP: 218.189.143.54, Vulnerable: False, Timestamp: 2024-03-06 08:51:16
IP: 14.215.125.26, Vulnerable: False, Timestamp: 2024-03-06 09:03:15
IP: 134.175.237.77, Vulnerable: False, Timestamp: 2024-03-06 09:14:15
